In [1]:
import pandas as pd
import logging
import networkx as nx
import time


from causal_reasoning.causal_model import CausalModel

In [2]:
class Query:
    def __init__(self, inter_label: str, inter_value: int, target_label: str, target_value: int, query_value: float) -> None:
        self.intervention_label = inter_label
        self.intervention_value = inter_value
        self.target_label = target_label
        self.target_value = target_value
        self.query = query_value

def run_queries(nodes_set: set, model: CausalModel, output_path: str):
    with open(output_path, 'w') as file:
        queries = []
        for intervention in nodes_set:
            for target in nodes_set:
                if intervention == target:
                    continue
                for intervention_value in range(2):
                    for target_value in range(2):
                        model.set_interventions([(intervention, intervention_value)])
                        model.set_target((target, target_value))
                        start = time.perf_counter()
                        query = model.inference_intervention_query()
                        end = time.perf_counter()
                        queries.append(Query(intervention, intervention_value, target, target_value, query))
                        file.write(f"P({target}={target_value}|do({intervention}={intervention_value}))\n")
                        file.write(f"{query[0]} <= P <= {query[1]}\n")
                        file.write(f"Inference time taken: {end-start:.6f} seconds\n")
                        file.write("---------------------------------------------------------\n")

## First Model

In [3]:
edges_list_1 = [
    ('NewDeploy', 'MemoryLeak'),
    ('MemoryLeak', 'MemoryUsageHigh'), 
    ("MemoryUsageHigh", "ServiceCrash"), 
    ("ServiceCrash", "OutageIncident"),
    ("HeavyTraffic", "MemoryUsageHigh"), 
    ("HeavyTraffic", "ServiceCrash"),
    # UNOBS
    ('Unob_helper_1', 'MemoryLeak'),
    ('Unob_helper_2', 'OutageIncident'),
    ('Unob_helper_3', 'NewDeploy'),
]
latent_nodes_1 = ['HeavyTraffic', 'Unob_helper_1', 'Unob_helper_2', 'Unob_helper_3']
nodes_set_1 = set()
for tuple in edges_list_1:
    if tuple[0] not in latent_nodes_1:
        nodes_set_1.add(tuple[0])
    if tuple[1] not in latent_nodes_1:
        nodes_set_1.add(tuple[1])

edges_1 = nx.DiGraph(edges_list_1)

df_small_scale_model = pd.read_csv("small_scale_outage_incident_seed42.csv", index_col=0)

In [4]:
model_1 = CausalModel(
    data=df_small_scale_model,
    edges=edges_1,
    unobservables_labels=latent_nodes_1,
)
run_queries(nodes_set_1, model_1, "output_small_scale_outage_incident_seed42.txt")

## Second Model


In [5]:
edges_list_2 = [
    ("DB_Change", "DB_Latency"),
    ("DB_Latency", "MS-B_Latency"), 
    ("MS-B_Latency", "MS-B_Error"), 
    ("MS-B_Latency", "MS-A_Latency"),
    ("MS-B_Error", "MS-A_Error"), 
    ("MS-A_Latency", "MS-A_Threads"), 
    ("MS-A_Threads", "MS-A_Crash"), 
    ("MS-A_Error", "Outage"), 
    ("MS-A_Crash", "Outage"), 
    ("HeavyTraffic", "DB_Latency"), 
    ("HeavyTraffic", "MS-A_Latency"),
    # UNOBS
    ('Unob_helper_1', 'DB_Change'),
    ('Unob_helper_2', 'MS-B_Latency'),
    ('Unob_helper_3', 'MS-B_Error'),
    ('Unob_helper_4', 'MS-A_Error'),
    ('Unob_helper_5', 'MS-A_Threads'),
    ('Unob_helper_6', 'MS-A_Crash'),
    ('Unob_helper_7', 'Outage'),
]

latent_nodes_2 = ['HeavyTraffic', 'Unob_helper_1', 'Unob_helper_2', 'Unob_helper_3', 'Unob_helper_4', 'Unob_helper_5', 'Unob_helper_6', 'Unob_helper_7']
nodes_set_2 = set()
for tuple in edges_list_2:
    if tuple[0] not in latent_nodes_2:
        nodes_set_2.add(tuple[0])
    if tuple[1] not in latent_nodes_2:
        nodes_set_2.add(tuple[1])

edges_2 = nx.DiGraph(edges_list_2)

df_medium_scale_incident = pd.read_csv("medium_scale_outage_incident_seed42.csv", index_col=0)

In [6]:
model_2 = CausalModel(
    data=df_medium_scale_incident,
    edges=edges_2,
    unobservables_labels=latent_nodes_2,
)
run_queries(nodes_set_2, model_2, "output_medium_scale_outage_incident_seed42.txt")